In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from PIL import Image
from skimage.transform import resize
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/DRIVE/data/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/DRIVE/data/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/DRIVE/data/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/DRIVE/data/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>)

In [3]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters=16, dropout=0.2):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    #conv_2 = build_block(conv_2, n_filters)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    #conv_4 = build_block(conv_4, n_filters * 2)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4)
    #conv_6 = build_block(conv_6, n_filters * 4)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)
    conv_8 = build_block(conv_7, n_filters * 8)
    #conv_8 = build_block(conv_8, n_filters * 8)
    pool_4 = tf.keras.layers.AveragePooling2D(padding='same')(conv_8)
    drop_4 = tf.keras.layers.SpatialDropout2D(dropout)(pool_4)
    
    conv_9 = build_block(drop_4, n_filters * 16)
    conv_10 = build_block(conv_9, n_filters * 16)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_10) #(-1, 16, 16, 256)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_8])#(-1, 16, 16, 384)
    upsp_1 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_1)#(-1, 16, 16, 384)
    conv_11 = build_block(upsp_1, n_filters * 8)#(-1, 16, 16, 128)
    conv_12 = build_block(conv_11, n_filters * 8)#(-1, 16, 16, 128)
    
    upsp_2 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_12) #(-1, 32, 32, 128)
    upsp_2 = tf.keras.layers.concatenate([upsp_2, conv_6]) #(-1, 32, 32, 192)
    upsp_2 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_2) #(-1, 32, 32, 192)
    conv_13 = build_block(upsp_2, n_filters * 4) #(-1, 32, 32, 64)
    conv_14 = build_block(conv_13, n_filters * 4) #(-1, 32, 32, 64)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_14) #(-1, 64, 64, 64)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 64, 64, 96)
    upsp_3 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_3) #(-1, 64, 64, 96)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 64, 64, 32)
    conv_16 = build_block(conv_15, n_filters * 2) #(-1, 64, 64, 32)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_16) #(-1, 128, 128, 32)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 128, 128, 48)
    upsp_4 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_4)#(-1, 128, 128, 48)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 128, 128, 16)
    conv_18 = build_block(conv_17, n_filters)#(-1, 128, 128, 16)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(conv_18)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

def random_crop(img, msk, random_crop_size):
    # Note: image_data_format is 'channel_last'
    #assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :], msk[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_x_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        batch_y_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        for i in range(batch_x.shape[0]):
            batch_x_crops[i], batch_y_crops[i] = random_crop(batch_x[i], batch_y[i], (crop_length, crop_length))
        yield (batch_x_crops, batch_y_crops)

In [4]:
model = build_unet(input_shape=(128, 128, 1))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 144         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 16) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [5]:
epochs, batch_size, lr, filepath = 10000, 4, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(horizontal_flip=True, vertical_flip=True) #width_shift_range=0.1, height_shift_range=0.1, 
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = crop_generator(zip(image_generator, mask_generator), 128)
test_generator = crop_generator(tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=batch_size), 128)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    #initial_epoch=65,
                    callbacks=[checkpoint, tb])

Epoch 1/10000
4/5 [=======================>......] - ETA: 4s - loss: 2.9071 - acc: 0.5047 
Epoch 00001: val_acc improved from -inf to 0.29874, saving model to day_1.weights.best.hdf5
5/5 [==============================] - 22s 4s/step - loss: 2.8523 - acc: 0.4979 - val_loss: 2.9286 - val_acc: 0.2987
Epoch 2/10000
4/5 [=======================>......] - ETA: 0s - loss: 2.4969 - acc: 0.6303
Epoch 00002: val_acc did not improve from 0.29874
5/5 [==============================] - 1s 129ms/step - loss: 2.4893 - acc: 0.6473 - val_loss: 3.6532 - val_acc: 0.0937
Epoch 3/10000
4/5 [=======================>......] - ETA: 0s - loss: 2.2443 - acc: 0.7283
Epoch 00003: val_acc did not improve from 0.29874
5/5 [==============================] - 1s 127ms/step - loss: 2.2038 - acc: 0.7370 - val_loss: 3.6287 - val_acc: 0.1056
Epoch 4/10000
4/5 [=======================>......] - ETA: 0s - loss: 2.0867 - acc: 0.7602
Epoch 00004: val_acc did not improve from 0.29874
5/5 [==============================] - 1s 

Epoch 32/10000
4/5 [=======================>......] - ETA: 0s - loss: 1.0962 - acc: 0.8730
Epoch 00032: val_acc did not improve from 0.84866
5/5 [==============================] - 1s 129ms/step - loss: 1.1016 - acc: 0.8728 - val_loss: 1.3855 - val_acc: 0.8460
Epoch 33/10000
4/5 [=======================>......] - ETA: 0s - loss: 1.0373 - acc: 0.8675
Epoch 00033: val_acc did not improve from 0.84866
5/5 [==============================] - 1s 133ms/step - loss: 1.0451 - acc: 0.8652 - val_loss: 1.3741 - val_acc: 0.8453
Epoch 34/10000
4/5 [=======================>......] - ETA: 0s - loss: 1.0765 - acc: 0.8761
Epoch 00034: val_acc improved from 0.84866 to 0.85721, saving model to day_1.weights.best.hdf5
5/5 [==============================] - 1s 173ms/step - loss: 1.0406 - acc: 0.8816 - val_loss: 1.2750 - val_acc: 0.8572
Epoch 35/10000
4/5 [=======================>......] - ETA: 0s - loss: 1.0264 - acc: 0.8695
Epoch 00035: val_acc did not improve from 0.85721
5/5 [=============================

4/5 [=======================>......] - ETA: 0s - loss: 0.7003 - acc: 0.8996
Epoch 00063: val_acc did not improve from 0.85993
5/5 [==============================] - 1s 125ms/step - loss: 0.7061 - acc: 0.8984 - val_loss: 1.3596 - val_acc: 0.8347
Epoch 64/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7970 - acc: 0.8768
Epoch 00064: val_acc did not improve from 0.85993
5/5 [==============================] - 1s 125ms/step - loss: 0.8015 - acc: 0.8730 - val_loss: 1.3261 - val_acc: 0.8467
Epoch 65/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7676 - acc: 0.8776
Epoch 00065: val_acc did not improve from 0.85993
5/5 [==============================] - 1s 125ms/step - loss: 0.7669 - acc: 0.8776 - val_loss: 1.3585 - val_acc: 0.8235
Epoch 66/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7324 - acc: 0.8895
Epoch 00066: val_acc did not improve from 0.85993
5/5 [==============================] - 1s 125ms/step - loss: 0.7077 - acc: 0.8921 - val_loss: 

Epoch 95/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7625 - acc: 0.8737
Epoch 00095: val_acc did not improve from 0.88102
5/5 [==============================] - 1s 127ms/step - loss: 0.7547 - acc: 0.8746 - val_loss: 0.9102 - val_acc: 0.8766
Epoch 96/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.6467 - acc: 0.8854
Epoch 00096: val_acc did not improve from 0.88102
5/5 [==============================] - 1s 128ms/step - loss: 0.7099 - acc: 0.8754 - val_loss: 0.9818 - val_acc: 0.8556
Epoch 97/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7031 - acc: 0.8823
Epoch 00097: val_acc did not improve from 0.88102
5/5 [==============================] - 1s 127ms/step - loss: 0.6921 - acc: 0.8828 - val_loss: 1.1016 - val_acc: 0.8378
Epoch 98/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7529 - acc: 0.8791
Epoch 00098: val_acc did not improve from 0.88102
5/5 [==============================] - 1s 127ms/step - loss: 0.7508 - acc: 0.87

Epoch 126/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7769 - acc: 0.8595
Epoch 00126: val_acc did not improve from 0.88794
5/5 [==============================] - 1s 128ms/step - loss: 0.7725 - acc: 0.8639 - val_loss: 1.0341 - val_acc: 0.8655
Epoch 127/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.7606 - acc: 0.8628
Epoch 00127: val_acc did not improve from 0.88794
5/5 [==============================] - 1s 128ms/step - loss: 0.7495 - acc: 0.8641 - val_loss: 0.8317 - val_acc: 0.8696
Epoch 128/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.6997 - acc: 0.8792
Epoch 00128: val_acc did not improve from 0.88794
5/5 [==============================] - 1s 128ms/step - loss: 0.6738 - acc: 0.8813 - val_loss: 0.6745 - val_acc: 0.8858
Epoch 129/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.5991 - acc: 0.8906
Epoch 00129: val_acc improved from 0.88794 to 0.90192, saving model to day_1.weights.best.hdf5
5/5 [=========================

...

4/5 [=======================>......] - ETA: 0s - loss: 0.4240 - acc: 0.8877
Epoch 09852: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.4069 - acc: 0.8932 - val_loss: 0.5066 - val_acc: 0.8878
Epoch 9853/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3637 - acc: 0.9046
Epoch 09853: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.3738 - acc: 0.9024 - val_loss: 0.5836 - val_acc: 0.8857
Epoch 9854/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3787 - acc: 0.8995
Epoch 09854: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 131ms/step - loss: 0.3682 - acc: 0.9026 - val_loss: 0.5520 - val_acc: 0.8903
Epoch 9855/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3761 - acc: 0.8990
Epoch 09855: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 131ms/step - loss: 0.3788 - acc: 0.9002 - val_

4/5 [=======================>......] - ETA: 0s - loss: 0.3955 - acc: 0.8958
Epoch 09883: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 131ms/step - loss: 0.4018 - acc: 0.8934 - val_loss: 0.4570 - val_acc: 0.9053
Epoch 9884/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3578 - acc: 0.9075
Epoch 09884: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.3869 - acc: 0.8985 - val_loss: 0.5927 - val_acc: 0.8888
Epoch 9885/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.4058 - acc: 0.8916
Epoch 09885: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 132ms/step - loss: 0.4071 - acc: 0.8915 - val_loss: 0.5534 - val_acc: 0.8841
Epoch 9886/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3922 - acc: 0.8989
Epoch 09886: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.3797 - acc: 0.9026 - val_

4/5 [=======================>......] - ETA: 0s - loss: 0.3737 - acc: 0.9006
Epoch 09914: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.3858 - acc: 0.8971 - val_loss: 0.5118 - val_acc: 0.8957
Epoch 9915/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3641 - acc: 0.9027
Epoch 09915: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 132ms/step - loss: 0.3543 - acc: 0.9056 - val_loss: 0.4786 - val_acc: 0.8972
Epoch 9916/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3542 - acc: 0.9110
Epoch 09916: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 131ms/step - loss: 0.3806 - acc: 0.9038 - val_loss: 0.5140 - val_acc: 0.8894
Epoch 9917/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3857 - acc: 0.8986
Epoch 09917: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 131ms/step - loss: 0.3819 - acc: 0.8991 - val_

4/5 [=======================>......] - ETA: 0s - loss: 0.3837 - acc: 0.8989
Epoch 09945: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 133ms/step - loss: 0.3574 - acc: 0.9059 - val_loss: 0.4713 - val_acc: 0.8983
Epoch 9946/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.4118 - acc: 0.8890
Epoch 09946: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 132ms/step - loss: 0.4131 - acc: 0.8888 - val_loss: 0.5410 - val_acc: 0.8847
Epoch 9947/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3305 - acc: 0.9133
Epoch 09947: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.3302 - acc: 0.9121 - val_loss: 0.4903 - val_acc: 0.8959
Epoch 9948/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3755 - acc: 0.8989
Epoch 09948: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 129ms/step - loss: 0.3788 - acc: 0.8991 - val_

4/5 [=======================>......] - ETA: 0s - loss: 0.3545 - acc: 0.9081
Epoch 09976: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 131ms/step - loss: 0.3590 - acc: 0.9058 - val_loss: 0.4986 - val_acc: 0.8960
Epoch 9977/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3919 - acc: 0.8970
Epoch 09977: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.4057 - acc: 0.8935 - val_loss: 0.4876 - val_acc: 0.8994
Epoch 9978/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3179 - acc: 0.9137
Epoch 09978: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 130ms/step - loss: 0.3223 - acc: 0.9132 - val_loss: 0.4308 - val_acc: 0.9104
Epoch 9979/10000
4/5 [=======================>......] - ETA: 0s - loss: 0.3929 - acc: 0.8953
Epoch 09979: val_acc did not improve from 0.92050
5/5 [==============================] - 1s 131ms/step - loss: 0.3890 - acc: 0.8954 - val_